<h1> MongoDB project </h1>

* mongoDB 사용하기 위해서는 [pymongo] 라이브러리를 설치해야 함.

In [1]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


* MongoDB 서버 연결하기

In [2]:
from pymongo import MongoClient
import gridfs
import os
from PIL import Image
import io

In [3]:
client = MongoClient("mongodb://localhost:27017/")  # 로컬 MongoDB

# 특정 데이터베이스 선택
db = client['myImageDatabase']

# 16MB 이상의 큰 파일을 저장할 수 있는 GridFs
# 대용량 파일을 효율적으로 저장 & 검색 가능

fs = gridfs.GridFS(db)

* image 저장, 검색, 삭제, 수정

In [4]:
# 1. 이미지 저장(추가) 함수
def save_image(image_path):
    """
    지정된 경로에서 이미지를 읽고 MongoDB에 저장하는 함수
    """
    if not os.path.exists(image_path):
        print("Error: File not found.")
        return
    
    filename = os.path.basename(image_path)
    with open(image_path, "rb") as img:
        file_id = fs.put(img, filename=filename)
        print(f"Image saved with ID: {file_id}")


# 2. 이미지 검색 함수
def list_images():
    """
    저장된 모든 이미지 파일명을 조회하는 함수
    """
    files = fs.find()
    image_list = [file.filename for file in files]
    
    if image_list:
        print("Stored Images:")
        for img in image_list:
            print(f"- {img}")
    else:
        print("No images found in the database.")

# 3. 이미지 삭제 함수
def delete_image(filename):
    """
    지정된 파일명을 가진 이미지를 MongoDB에서 삭제하는 함수
    """
    file_data = fs.find_one({"filename": filename})
    if file_data:
        fs.delete(file_data._id)
        print(f"Deleted image: {filename}")
    else:
        print("Error: Image not found.")


# 4. 이미지 수정 함수
def update_image(filename, new_image_path):
    """
    기존의 이미지를 삭제하고 새로운 이미지로 업데이트하는 함수
    """
    if not os.path.exists(new_image_path):
        print("Error: New image file not found.")
        return
    
    delete_image(filename)
    save_image(new_image_path)


# 5. 이미지 다운로드 함수
def download_image(filename, output_path):
    """
    MongoDB에서 이미지를 다운로드하여 로컬에 저장하는 함수
    """
    file_data = fs.find_one({"filename": filename})
    if file_data:
        with open(output_path, "wb") as output:
            output.write(file_data.read())
        print(f"Image downloaded successfully: {output_path}")
    else:
        print("Error: Image not found.")

* 실행 함수 ! __ !

In [7]:
def main():
    """
    사용자가 실행하는 메인 메뉴 함수
    원하는 기능을 선택하여 실행할 수 있습니다.
    """
    while True:
        print("\n===== MongoDB Image Manager =====")
        print("1. 이미지 저장 (Save Image)")
        print("2. 저장된 이미지 목록 확인 (List Images)")
        print("3. 이미지 삭제 (Delete Image)")
        print("4. 이미지 수정 (Update Image)")
        print("5. 이미지 다운로드 (Download Image)")
        print("6. 프로그램 종료 (Exit)")
        print("===== ===== ===== ===== ===== =====")
        
        choice = input(">> 원하는 작업을 선택하세요 (Enter your choice): ")
        
        if choice == "1":
            image_path = input(">> 저장할 이미지 경로를 입력하세요 (Enter image path to save): ")
            save_image(image_path)
        elif choice == "2":
            list_images()
        elif choice == "3":
            filename = input(">> 삭제할 이미지 파일명을 입력하세요 (Enter filename to delete): ")
            delete_image(filename)
        elif choice == "4":
            filename = input(">> 수정할 이미지 파일명을 입력하세요 (Enter filename to update): ")
            new_image_path = input(">> 새로운 이미지 경로를 입력하세요 (Enter new image path): ")
            update_image(filename, new_image_path)
        elif choice == "5":
            filename = input(">> 다운로드할 이미지 파일명을 입력하세요 (Enter filename to download): ")
            output_path = input(">> 이미지를 저장할 경로를 입력하세요 (Enter output path): ")
            download_image(filename, output_path)
        elif choice == "6":
            print(">> 프로그램을 종료합니다. (Exiting...)")
            break
        else:
            print(">> 잘못된 입력입니다. 다시 시도하세요. (Invalid choice. Please try again.)")

if __name__ == "__main__":
    main()



===== MongoDB Image Manager =====
1. 이미지 저장 (Save Image)
2. 저장된 이미지 목록 확인 (List Images)
3. 이미지 삭제 (Delete Image)
4. 이미지 수정 (Update Image)
5. 이미지 다운로드 (Download Image)
6. 프로그램 종료 (Exit)
===== ===== ===== ===== ===== =====
Stored Images:
- 쳇바퀴타는햄스터.jfif
- 밥먹는햄스터.jfif
- 햄스터.jfif
- 밥먹는햄스터.jfif
- 햄스터.jfif
- 햄스터.jfif
- 밥먹는햄스터.jfif
- 쳇바퀴타는햄스터.jfif

===== MongoDB Image Manager =====
1. 이미지 저장 (Save Image)
2. 저장된 이미지 목록 확인 (List Images)
3. 이미지 삭제 (Delete Image)
4. 이미지 수정 (Update Image)
5. 이미지 다운로드 (Download Image)
6. 프로그램 종료 (Exit)
===== ===== ===== ===== ===== =====
Deleted image: 햄스터.jfif

===== MongoDB Image Manager =====
1. 이미지 저장 (Save Image)
2. 저장된 이미지 목록 확인 (List Images)
3. 이미지 삭제 (Delete Image)
4. 이미지 수정 (Update Image)
5. 이미지 다운로드 (Download Image)
6. 프로그램 종료 (Exit)
===== ===== ===== ===== ===== =====
Stored Images:
- 쳇바퀴타는햄스터.jfif
- 밥먹는햄스터.jfif
- 밥먹는햄스터.jfif
- 햄스터.jfif
- 햄스터.jfif
- 밥먹는햄스터.jfif
- 쳇바퀴타는햄스터.jfif

===== MongoDB Image Manager =====
1. 이미지 저장 (Save Image)
2. 저장된 이미지 목록 확인 (L